In [ ]:
from notebooks import *

In [ ]:
print('\n'.join(
    'td .label-i.i-%-2s { color: white; background: %s; }' % (i, mpl.colors.rgb2hex(x))
    for i, x in enumerate(take(50, cycle(chain(
        islice(mpl.cm.tab20.colors, 0, None, 2),
        islice(mpl.cm.tab20.colors, 1, None, 2),
        mpl.cm.Pastel1.colors,
    ))))
))

In [ ]:
# Debug mobile cosine_distance
from notebooks import *
sg.init(None)
recs = sg.search_recs
q = recs.query('xc_id == 417559').iloc[0]
s = recs.query('xc_id == 416810').iloc[0]
display(
    sk.metrics.pairwise.distance_metrics()['cosine']([s.f_preds], [q.f_preds]),
    np.dot(s.f_preds, q.f_preds) / np.linalg.norm(s.f_preds) / np.linalg.norm(q.f_preds),
    1 - np.dot(s.f_preds, q.f_preds) / np.linalg.norm(s.f_preds) / np.linalg.norm(q.f_preds),
)

In [ ]:
# Find search.classes_ for mobile
from notebooks import *
sg.init(None)
display(
    sk_dir_attrs(sg.search),
    sg.search.classes_,
)

In [ ]:
S = np.array([[1,2,3],[4,5,6]])
np.maximum(2.5, S)

In [ ]:
np.exp(2.0)
np.log(2.0)
np.log10(2.0)

In [ ]:
np.zeros((3,5))

In [ ]:
xs = np.array([1, 2, 4, 7, 0])
display(
    np.diff(xs),
    xs[1:] - xs[:-1],
)

In [ ]:
np.multiply.outer([1,2], [-3,-5,-7])

In [ ]:
np.zeros((3,5))
np.zeros((3,5))[0]
np.zeros((3,5))[:, 0]

In [ ]:
n_mels = 3
cols   = 5
weights = (np.random.rand(n_mels, cols) * 10).astype(int); weights
enorm = (np.random.rand(n_mels) * 10).astype(int); enorm
weights.shape
enorm[:, np.newaxis].shape
enorm[:, np.newaxis]
np.broadcast_to(enorm[:, np.newaxis], (3,5))
weights * enorm[:, np.newaxis]

In [ ]:
weights
weights**2
weights*weights
np.dot(weights, weights.T)

In [ ]:
S = np.array([[1,2,3],[4,5,6]]); S
S[0] * S[1]

In [ ]:
scipy.signal.hann(5)

In [ ]:
S = np.array([[1,2,3],[4,5,6]]); S
S**2
(S**2).mean()
S / np.sqrt((S ** 2).mean())

In [ ]:
# xs_count   = 5286
xs_count   = 5120
nperseg    = 512
noverlap   = 256
win_length = nperseg
hop_length = noverlap
f          = win_length
t_         = (xs_count - win_length) / hop_length + 1
t          = int(t_)
hops       = list(range(xs_count)[0 : xs_count - win_length : hop_length])
dict(
    xs_count=xs_count,
    win_length=win_length,
    hop_length=hop_length,
    f=f,
    t_=t_,
    t=t,
    len_hops=len(hops),
    hops=hops,
)

In [ ]:
for n, w, h, exp in [
    (14, 6, 3, 3),
    (15, 6, 3, 4),
    (16, 6, 3, 4),
    (17, 6, 3, 4),
    (18, 6, 3, 5),
    (19, 6, 3, 5),
    (5210, 512, 256, 0),
]:
    xs = range(n)
    segs = list([x, x+w] for x in xs[0 : n-w : h])
    t = (n - w) // h + 1
    n_segs = len(segs)
    display({
        'n, w, h': (n, w, h),
        't, n_segs': (t, n_segs),
        # 'n/h': round(n/h, 3),
        # '(n-w)/h': round((n-w)/h, 3),
        # '(n-w)/h+1': round((n-w)/h+1, 3),
        # '[0:n:h]': list([x, x+w] for x in xs[0 : n : h]),
        # '[0:n-w:h]': list([x, x+w] for x in xs[0 : n-w : h]),
    })

In [ ]:
# NOTE Same as swift stride(from:through:by:), not stride(from:to:by:)
len(range(5210)[0 : 5210 - 512 : 256])

In [ ]:
import librosa
pp(librosa.fft_frequencies(sr=22050, n_fft=512).shape)
pp(librosa.fft_frequencies(sr=22050, n_fft=512)[250:])

In [ ]:
np.linspace(0, 10, 5)

In [ ]:
import librosa
mel_basis = librosa.filters.mel(sr=22050, n_fft=512, n_mels=40)
mel_basis.shape

In [ ]:
M = lambda r, c: np.random.rand(r, c)
np.dot(M(1,2), M(2,0))
np.dot(M(0,2), M(2,3))
np.dot(M(1,0), M(0,3))

In [ ]:
np.abs(np.fft.rfft(np.array([1])))
np.abs(np.fft.rfft(np.array([1,1])))
np.abs(np.fft.rfft(np.array([1,1,1])))
np.abs(np.fft.rfft(np.array([1,1,1,1])))

In [ ]:
x = np.random.rand(5_000_000) * 100
X = np.random.rand(3,5)
np.save('/tmp/array.npy',  x)
np.save('/tmp/matrix.npy', X)
np.savez('/tmp/z.npz', x=x, X=X)
np.savez_compressed('/tmp/z_compressed.npz', x=x, X=X)
display(
    np.load('/tmp/array.npy'),
    np.load('/tmp/matrix.npy'),
    dict(np.load('/tmp/z.npz')),
    dict(np.load('/tmp/z_compressed.npz')),
)

In [ ]:
X = np.random.rand(3,5); X
X.mean(axis=1)

In [ ]:
np.load(
    '/Users/danb/hack/bubo/data/'
    'cache/payloads/search_recs-version[8],limit[100],audio_s[10],countries_k[na],com_names_k[ca],num_recs[None]-fb42447/mobile-version[1]/search_recs/models/'
    'classifier_.estimators_[97].coef_.npy'
).shape

In [ ]:
obj = dict(
    a=1,
    b='foo',
    c=[1,2,3],
)
np.save('/tmp/obj.npy', obj)
np.load('/tmp/obj.npy')

In [ ]:
from notebooks import *
sg.init(None)

In [ ]:
# For sk.linear_model.LogisticRegressionOneClass
X = np.random.rand(5,1500)
display(
    sg.search.classifier_.predict_proba(X).shape,
    sg.search.classifier_.estimators_[0].predict_proba(X).shape,
    sg.search.classifier_.estimators_[0].predict_proba(X),
    scipy.special.expit(sg.search.classifier_.estimators_[0].decision_function(X)),
)

In [ ]:
sg.search.classifier_.estimators_[0].coef_.shape
sg.search.classifier_.estimators_[0].intercept_.shape
sk_dir_attrs(sg.search.classifier_)
sg.search.classifier_.estimators_[0]
sg.search.classifier_.intercept_.shape
sg.search.classifier_.estimators_[0].intercept_.shape
sk_dirs_attrs(sg.search.classifier_)

In [ ]:
from notebooks import *
import pprint
(xc.metadata
    [lambda df: pd.notnull(df.species)]
    [lambda df: df.country == 'Costa Rica']
    .pipe(df_remove_unused_categories).com_name.sort_values()
    .pipe(puts, f=lambda s: len(s.unique()))
    # .pipe(lambda s:
    #     pprint.pprint(s.unique().tolist(), indent=4, width=120, compact=True)
    # )
);

In [ ]:
# Q: What kinds of huge recs are out there?
#   - A: Holy crap 1.4gb!
# Like _compute_search_recs
recs = sg.xc_meta
# Like recs_featurize_metdata_audio_slice
recs = (recs
    .pipe(recs_featurize_metadata)
)
display(
    recs.shape,
    recs.samples_mb[lambda s: s > 100].shape,
)
recs.samples_mb[lambda s: s >= 0].plot.hist(bins=100, logy=True)

In [ ]:
from notebooks import *
load = Load()
load.recs_paths(['xc'])[:3]

In [ ]:
from notebooks import *
load = Load()
with cache_control(enabled=False):
    display(
        load.recs(paths=[('xc', '/Users/danb/hack/bubo/features/data/xc/data/ABAN/12046/audio.mp3')]),
    )